In [32]:
spark

In [33]:
from pyspark import SparkConf

In [44]:
spark.conf.set('spark.executor.cores', 34)
spark.conf.set('spark.driver.cores', 2)
spark.conf.set('spark.driver.memory', '5g')
spark.conf.set('spark.executor.memory', '55g')
spark.conf.set('spark.cores.max', 34)
spark.conf.set('spark.default.parallelism', 34)

In [35]:
# SparkConf()
# conf = SparkConf()
# # conf.set("spark.storage.memoryFraction", "0.5")
# sc = SparkContext(conf=conf)

In [36]:
import pyspark
from pyspark import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import TimestampType

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import pandas as pd
from sklearn import *
from sklearn import preprocessing
# import lightgbm as lgb
from multiprocessing import *
from sklearn.model_selection import StratifiedKFold

pd.set_option('display.max_columns', None)

# sc = pyspark.SparkContext('local[*]')
sc=spark
sqlContext = SQLContext(sparkContext=sc)

#sc._conf.getAll()

In [37]:
s3_path = "/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11"
out_path = "/home/jovyan/work/sparkling-water-2.2.2/work/s3.flight.pq.11.comb"

In [38]:
!pwd

/home/jovyan/work/sparkling-water-2.2.2/work/flight/prod


In [39]:
! ls /home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11

flight_10_1   flight_13_35  flight_22_11  flight_33_6	flight_6_18
flight_10_14  flight_13_36  flight_22_12  flight_33_7	flight_6_19
flight_10_15  flight_13_37  flight_22_13  flight_33_8	flight_6_2
flight_10_16  flight_13_38  flight_22_14  flight_33_9	flight_6_20
flight_10_17  flight_13_39  flight_22_5   flight_34_10	flight_6_21
flight_10_18  flight_13_4   flight_22_6   flight_34_11	flight_6_22
flight_10_19  flight_13_40  flight_22_7   flight_34_12	flight_6_23
flight_10_2   flight_13_9   flight_22_8   flight_34_13	flight_6_24
flight_10_20  flight_14_1   flight_22_9   flight_34_14	flight_6_25
flight_10_21  flight_14_10  flight_23_10  flight_34_5	flight_6_26
flight_10_22  flight_14_11  flight_23_11  flight_34_6	flight_6_27
flight_10_23  flight_14_12  flight_23_12  flight_34_7	flight_6_28
flight_10_24  flight_14_13  flight_23_13  flight_34_8	flight_6_29
flight_10_25  flight_14_15  flight_23_14  flight_34_9	flight_6_3
flight_10_26  flight_14_16  flight_23_5   flight_3_5	flight_6_30
flight_1

In [40]:
! ls /home/jovyan/work/sparkling-water-2.2.2/work/s3.flight.pq.11.comb

fromCity=Hangzhou  fromCity=Singapore  fromCity=Taipei	_SUCCESS


In [41]:
from glob import glob

folders = glob(s3_path+"/*/")

try:
    folders.remove(s3_path+"/pq/") #pq/flight_10_31 Is this duplicate?
except ValueError:
    print("error")

In [42]:
folders[0:10]

['/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_13_16/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_23_14/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_14_36/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_24_7/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_12_33/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_8_34/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_13_2/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_10_19/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_25_11/',
 '/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11/flight_2_13/']

In [43]:
%%time

sample_prop = 1

import time

start = time.time()

for i, f in enumerate(folders[0:3]):
    end = time.time()
    elapsed_min = (end - start)/60
    print("Processing: " + str(i) + "/" + str(len(folders)) + "  " + 
          f.replace(s3_path, "").replace("/", "") + " Elapsed time: " + str(round(elapsed_min, 0)) + " minutes")
    
    if (sample_prop == 1): #no sample
        df = (sqlContext.read.parquet(f)          
                .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))    
    else:    
        df = (sqlContext.read.parquet(f)
                .sample(False, sample_prop, seed=0)
                .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))    
        
    if (i == 0):        
        df.coalesce(2).write.partitionBy("fromCity", "toCity").mode("overwrite").save(out_path)
    else:        
        df.coalesce(2).write.partitionBy("fromCity", "toCity").mode("append").save(out_path)            

Processing: 0/622  flight_13_16 Elapsed time: 0.0 minutes
Processing: 1/622  flight_23_14 Elapsed time: 1.0 minutes
Processing: 2/622  flight_14_36 Elapsed time: 3.0 minutes
CPU times: user 48 ms, sys: 28 ms, total: 76 ms
Wall time: 2min 54s


In [45]:
df = sqlContext.read.parquet(out_path)

In [46]:
df.count()

1828314

In [47]:
lookforwardDays = 19
ndays = lambda i: i * 86400 # number of seconds in a day
byVar = ['toCity', 'depDate', 'stayDays', 'timeline1_departureTime_time', 'timeline2_departureTime_time',
         'timeline1_carrier_airlineCode', 'timeline2_carrier_airlineCode']

w = (Window.partitionBy('routeCombKey')
     .orderBy(F.col('searchDate').cast('timestamp').cast('long'))
     .rowsBetween(0, ndays(lookforwardDays)))

threshold = 20

udfGetWeekNumer = lambda dt: F.udf(dt.isocalendar()[1], IntegerType())
# udfGetAvg = lambda x: F.udf(sum(x)/float(len(x)))\n\n

df2 = (df.filter(df.price > 0)
       .withColumn('routeCombKey', F.concat_ws('-', F.col('fromCity'),
                                   F.col('toCity'),
                                   F.col('stayDays'), F.col('depDate'),
                                   F.col('timeline1_departureTime_time'),
                                   F.col('timeline2_departureTime_time'),
                                   F.col('timeline1_carrier_airlineCode'),
                                   F.col('timeline2_carrier_airlineCode')))
                                   # join_udf(F.col('timeline1_departureTime_time')),
                                   # join_udf(F.col('timeline2_departureTime_time')),
                                   # join_udf(F.col('timeline1_carrier_airlineCode')),
                                   # join_udf(F.col('timeline2_carrier_airlineCode'))))
       .withColumn('futureMinPrice', F.min(F.col('price')).over(w))
       .withColumn('priceWillDrop', (F.col('price') - F.col('futureMinPrice') > threshold).cast('int'))
       .withColumn('saving0', F.col('price') - F.col('futureMinPrice'))
       .withColumn('temp0', F.lit(0))
       .withColumn('saving', F.greatest('saving0', 'temp0'))
       .drop('saving0', 'temp0')
       .withColumn('leadTime', F.datediff(F.col('depDate'), F.col('searchDate')))
       .withColumn('depWeekOfYear', F.weekofyear(F.col('depDate')))
       .withColumn('retWeekOfYear', F.weekofyear(F.col('leg2_departureTime_date')))
       .withColumn('depMonth', F.month(F.col('depDate')))
       .withColumn('depDayOfMonth', F.dayofmonth(F.col('depDate')))
       .withColumn('depDayOfYear', F.dayofyear(F.col('depDate')))
       .withColumn('depDayOfWeek', F.date_format(F.col('depDate'), 'EEEE'))
       .withColumn('retDayOfWeek', F.date_format(F.col('leg2_departureTime_date'), 'EEEE'))
       .withColumn('searchDayOfWeek', F.date_format(F.col('searchDate'), 'EEEE'))
       .withColumn('leg1_noOfTicketsLeft', 
                   F.when(F.col('leg1_carrierSummary_noOfTicketsLeft') > 0, 
                          F.col('leg1_carrierSummary_noOfTicketsLeft')).otherwise(99))
        .withColumn('leg2_noOfTicketsLeft', 
                    F.when(F.col('leg2_carrierSummary_noOfTicketsLeft') > 0, 
                           F.col('leg2_carrierSummary_noOfTicketsLeft')).otherwise(99))
       .withColumn('leg1_cabinClass_0', F.col('timeline1_carrier_cabinClass').getItem(0))
       .withColumn('leg1_cabinClass_1', F.col('timeline1_carrier_cabinClass').getItem(1))
       .withColumn('leg1_cabinClass_2', F.col('timeline1_carrier_cabinClass').getItem(2))
       .withColumn('leg2_cabinClass_0', F.col('timeline2_carrier_cabinClass').getItem(0))
       .withColumn('leg2_cabinClass_1', F.col('timeline2_carrier_cabinClass').getItem(1))
       .withColumn('leg2_cabinClass_2', F.col('timeline2_carrier_cabinClass').getItem(2))
       .select('price', 'priceWillDrop', 'futureMinPrice', 'saving',
               'fromCity', 'toCity',
               'searchDate',
               'routeCombKey',
               'leadTime', 
               'leg1_stops', 'leg2_stops',
               'leg1_noOfTicketsLeft', 'leg2_noOfTicketsLeft',
               'leg1_carrierSummary_airlineName', 'leg2_carrierSummary_airlineName',
               'leg1_departureTime_hour', 'leg2_departureTime_hour',
               'depWeekOfYear', 'depDayOfWeek','retWeekOfYear', 'retDayOfWeek',
               'searchDayOfWeek',
               'leg1_cabinClass_0', 'leg1_cabinClass_1', 'leg1_cabinClass_2',
               'leg2_cabinClass_0', 'leg2_cabinClass_1', 'leg2_cabinClass_2',
               'trip')
      )

# df2.cache()

In [49]:
df2.limit(100).toPandas()

,price,priceWillDrop,futureMinPrice,saving,fromCity,toCity,searchDate,routeCombKey,leadTime,leg1_stops,leg2_stops,leg1_noOfTicketsLeft,leg2_noOfTicketsLeft,leg1_carrierSummary_airlineName,leg2_carrierSummary_airlineName,leg1_departureTime_hour,leg2_departureTime_hour,depWeekOfYear,depDayOfWeek,retWeekOfYear,retDayOfWeek,searchDayOfWeek,leg1_cabinClass_0,leg1_cabinClass_1,leg1_cabinClass_2,leg2_cabinClass_0,leg2_cabinClass_1,leg2_cabinClass_2,trip
0,365.7,0,365.7,0.0,Hangzhou,Chiang Mai,2017-06-23,Hangzhou-Chiang Mai-0-2017-06-26-11:30am-10:00...,3,2,1,5,5,China Eastern Airlines,Xiamen Airlines,11,13,26,Monday,None,None,Friday,3,3,3,3,3,None,1
1,366.8,0,366.8,0.0,Hangzhou,Chiang Mai,2017-06-25,Hangzhou-Chiang Mai-0-2017-06-26-11:30am-10:00...,1,2,1,5,5,China Eastern Airlines,Xiamen Airlines,11,13,26,Monday,None,None,Sunday,3,3,3,3,3,None,1
2,1229.7,0,1229.7,0.0,Hangzhou,Chiang Mai,2017-06-23,Hangzhou-Chiang Mai-0-2017-06-26-8:10am-1:15pm...,3,2,1,9,5,,Xiamen Airlines,8,13,26,Monday,None,None,Friday,3,3,3,3,3,None,1
3,1231.8,0,1231.8,0.0,Hangzhou,Chiang Mai,2017-06-25,Hangzhou-Chiang Mai-0-2017-06-26-8:10am-1:15pm...,1,2,1,9,5,,Xiamen Airlines,8,13,26,Monday,None,None,Sunday,3,3,3,3,3,None,1
4,365.7,0,365.7,0.0,Hangzhou,Chiang Mai,2017-06-23,Hangzhou-Chiang Mai-0-2017-06-28-3:37pm-7:10pm...,5,2,1,5,5,China Eastern Airlines,Xiamen Airlines,15,13,26,Wednesday,None,None,Friday,3,3,3,3,3,None,1
5,366.8,0,366.8,0.0,Hangzhou,Chiang Mai,2017-06-25,Hangzhou-Chiang Mai-0-2017-06-28-3:37pm-7:10pm...,3,2,1,5,5,China Eastern Airlines,Xiamen Airlines,15,13,26,Wednesday,None,None,Sunday,3,3,3,3,3,None,1
6,646.5,0,646.5,0.0,Hangzhou,Chiang Mai,2017-06-23,Hangzhou-Chiang Mai-0-2017-06-29-10:30pm-8:40a...,6,1,1,1,5,China Southern Airlines,Xiamen Airlines,22,13,26,Thursday,None,None,Friday,3,3,None,3,3,None,1
7,671.5,1,646.5,25.0,Hangzhou,Chiang Mai,2017-06-25,Hangzhou-Chiang Mai-0-2017-06-29-10:30pm-8:40a...,4,1,1,9,5,China Southern Airlines,Xiamen Airlines,22,13,26,Thursday,None,None,Sunday,3,3,None,3,3,None,1
8,646.5,0,646.5,0.0,Hangzhou,Chiang Mai,2017-06-27,Hangzhou-Chiang Mai-0-2017-06-29-10:30pm-8:40a...,2,1,1,1,5,China Southern Airlines,Xiamen Airlines,22,13,26,Thursday,None,None,Tuesday,3,3,None,3,3,None,1
9,1413.6,1,1381.8,31.8,Hangzhou,Chiang Mai,2017-06-23,Hangzhou-Chiang Mai-0-2017-07-04-8:00am-12:00p...,11,2,1,4,5,Cathay Pacific,Xiamen Airlines,8,13,27,Tuesday,None,None,Friday,3,3,3,3,3,None,1


In [50]:
%%time
# Slow!

df3 = df2.filter(F.col('searchDate') <= F.lit('2017-06-23').cast(TimestampType()))
df3 = df3.withColumn('randVar', F.round(F.rand()*10, 0))
# df3.filter(F.col('searchDate') >= F.lit('2017-06-16').cast(TimestampType())).toPandas().to_pickle("D:\flight.pq.11\df3_test.pkl")
df3.groupby('randVar').agg(F.count('randVar').alias('count')).show(100)

# listRand = [x['randVar'] for x in df3.select('randVar').distinct().collect()]

# dfArray = [df3.where(F.col('randVar') == x) for x in listRand]

# # spark.conf.set("spark.driver.maxResultSize", "10g")

# i = 0

# for df_temp in dfArray:
#     df_temp.toPandas().to_pickle('../out_data/flight.pq.11_v2_pickle/df3_test_' + str(listRand[i]) + '.pkl')
#     i += 1

+-------+-----+
|randVar|count|
+-------+-----+
|    8.0|14875|
|    0.0| 7652|
|    7.0|14852|
|    1.0|14969|
|    4.0|15019|
|    3.0|14926|
|    2.0|14914|
|   10.0| 7482|
|    6.0|14949|
|    5.0|14932|
|    9.0|14664|
+-------+-----+

CPU times: user 8 ms, sys: 12 ms, total: 20 ms
Wall time: 23.6 s


In [51]:
df3.show()

+------+-------------+--------------+------------------+--------+----------+----------+--------------------+--------+----------+----------+--------------------+--------------------+-------------------------------+-------------------------------+-----------------------+-----------------------+-------------+------------+-------------+------------+---------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----+-------+
| price|priceWillDrop|futureMinPrice|            saving|fromCity|    toCity|searchDate|        routeCombKey|leadTime|leg1_stops|leg2_stops|leg1_noOfTicketsLeft|leg2_noOfTicketsLeft|leg1_carrierSummary_airlineName|leg2_carrierSummary_airlineName|leg1_departureTime_hour|leg2_departureTime_hour|depWeekOfYear|depDayOfWeek|retWeekOfYear|retDayOfWeek|searchDayOfWeek|leg1_cabinClass_0|leg1_cabinClass_1|leg1_cabinClass_2|leg2_cabinClass_0|leg2_cabinClass_1|leg2_cabinClass_2|trip|randVar|
+------+-------------+--------